전처리 정리
1. df19.csv, df20.csv, df21.csv, df22.csv, df23.csv 전부 합치기
2. 년, 월, 일, 시간, 분, 요일 칼럼 만들기
3. 2018년부터 2023년 데이터 추출
4. '사고자학년'결측값 제거
5. 나이 칼럼 만들어주기

# Module Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os 
from scipy import stats

import matplotlib.font_manager as fm

from datetime import datetime
# 설치된 폰트 출력
font_list = [font.name for font in fm.fontManager.ttflist]
plt.rcParams['font.family'] = 'NanumGothic' # 한글깨짐 방지

# Data Load

In [14]:
df19 = pd.read_csv('./../dataset/after_remove_null_data/df19.csv', index_col = 0)
df20 = pd.read_csv('./../dataset/after_remove_null_data/df20.csv', index_col = 0)
df21 = pd.read_csv('./../dataset/after_remove_null_data/df21.csv', index_col = 0)
df22 = pd.read_csv('./../dataset/after_remove_null_data/df22.csv', index_col = 0)
df23 = pd.read_csv('./../dataset/after_remove_null_data/df23.csv', index_col = 0)

In [15]:
df23.rename(columns={'매개물' : '사고매개물'}, inplace=True)


## 데이터 합치기

In [16]:
df = pd.concat([df19, df20, df21, df22, df23])
df.shape

(614977, 17)

## 년,월,일,시간,분,요일 칼럼 만들기

In [17]:
# 데이터 합친 후에 진행
## 'year', 'month', 'day', 'hour', 'min' 칼럼 새로 만들어주기
df['year'] = df['사고발생일'].str.split('-').str.get(0).astype(int)
df['month'] = df['사고발생일'].str.split('-').str.get(1).astype(int)
df['day'] = df['사고발생일'].str.split('-').str.get(2).astype(int)
df['hour'] = df['사고발생시각'].str.split(':').str.get(0).astype(int)
df['min'] = df['사고발생시각'].str.split(':').str.get(1).astype(int)

def get_day_of_week(date_str):
    # 월(0), 화(1), 수(2), 목(3), 금(4), 토(5), 일(6)
    date_format = '%Y-%m-%d'
    try:
        date_obj = datetime.strptime(date_str, date_format)
        day_of_week = date_obj.weekday()    
        return day_of_week

    except ValueError:
        return '올바른 날짜 형식이 아닙니다.'

df['week_day'] = df['사고발생일'].apply(get_day_of_week)



## 2018년부터 2023년 데이터만 추출

In [20]:
df = df[df['year'].isin([2018, 2019, 2020, 2021, 2022, 2023])]
df.shape

(614601, 23)

## '사고자학년' 결측값 제거

In [21]:
df = df[df['사고자학년'].notnull()]
df.shape

(614401, 23)

## 나이 칼럼 만들기

In [22]:
under_elemt_df = df[df['학교급'] == '유치원']
elemt_df = df[(df['학교급'] == '초등학교') & (df['사고자학년'].isin(['1학년', '2학년', '3학년', '4학년', '5학년', '6학년']))]
middle_school_df = df[(df['학교급'] == '중학교') & (df['사고자학년'].isin(['1학년', '2학년', '3학년']))]
high_school_df = df[(df['학교급'] == '고등학교') & (df['사고자학년'].isin(['1학년', '2학년', '3학년']))]

df_new = pd.concat([under_elemt_df, elemt_df, middle_school_df, high_school_df])
df_new.shape

(609988, 23)

In [23]:
df_new.head()

,구분,학교급,지역,교육청,설립유형,사고자구분,사고자성별,사고자학년,사고발생일,사고발생요일,...,사고부위,사고형태,사고당시활동,사고매개물,year,month,day,hour,min,week_day
28,A0000029,유치원,경기,용인교육지원청,공립,일반학생,여,유아,2018-12-28,금,...,머리(두부),물리적힘 노출,보행/주행,건물(문/창문/바닥/벽 등),2018,12,28,12,30,4
33,A0000034,유치원,경기,광주하남교육지원청,사립,일반학생,남,유아,2018-12-17,월,...,팔,기타,장난/놀이,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),2018,12,17,11,20,0
38,A0000039,유치원,경북,포항교육지원청,사립,일반학생,남,유아,2018-12-28,금,...,머리(두부),낙상-넘어짐,기타운동,건물(문/창문/바닥/벽 등),2018,12,28,9,20,4
39,A0000040,유치원,전북,익산교육지원청,공립,일반학생,남,유아,2018-12-26,수,...,치아(구강),물리적힘 노출,보행/주행,가구(책상/의자/책장/탁자/침대 등),2018,12,26,11,30,2
41,A0000042,유치원,부산,남부교육지원청,공립,일반학생,남,유아,2018-12-26,수,...,팔,낙상-넘어짐,장난/놀이,운동(놀이)용 장비/기구(공/운동기구/운동장 기구 등),2018,12,26,14,40,2


In [24]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Index: 609988 entries, 28 to 193169
Data columns (total 23 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   구분        609988 non-null  object
 1   학교급       609988 non-null  object
 2   지역        609988 non-null  object
 3   교육청       609988 non-null  object
 4   설립유형      609988 non-null  object
 5   사고자구분     609988 non-null  object
 6   사고자성별     609988 non-null  object
 7   사고자학년     609988 non-null  object
 8   사고발생일     609988 non-null  object
 9   사고발생요일    609988 non-null  object
 10  사고발생시각    609988 non-null  object
 11  사고시간      609988 non-null  object
 12  사고장소      609988 non-null  object
 13  사고부위      609988 non-null  object
 14  사고형태      609988 non-null  object
 15  사고당시활동    609988 non-null  object
 16  사고매개물     609988 non-null  object
 17  year      609988 non-null  int32 
 18  month     609988 non-null  int32 
 19  day       609988 non-null  int32 
 20  hour      609988 non-null  int

In [25]:

def make_age(row):
    # row : Series
    if (row['학교급'] == '유치원') and (row['사고자학년'] == '유아'):
        return 7
    elif (row['학교급'] == '초등학교') and (row['사고자학년'] == '1학년'):
        return 8
    elif (row['학교급'] == '초등학교') and (row['사고자학년'] == '2학년'):
        return 9
    elif (row['학교급'] == '초등학교') and (row['사고자학년'] == '3학년'):
        return 10
    elif (row['학교급'] == '초등학교') and (row['사고자학년'] == '4학년'):
        return 11
    elif (row['학교급'] == '초등학교') and (row['사고자학년'] == '5학년'):
        return 12
    elif (row['학교급'] == '초등학교') and (row['사고자학년'] == '6학년'):
        return 13
    elif (row['학교급'] == '중학교') and (row['사고자학년'] == '1학년'):
        return 14
    elif (row['학교급'] == '중학교') and (row['사고자학년'] == '2학년'):
        return 15
    elif (row['학교급'] == '중학교') and (row['사고자학년'] == '3학년'):
        return 16
    elif (row['학교급'] == '고등학교') and (row['사고자학년'] == '1학년'):
        return 17
    elif (row['학교급'] == '고등학교') and (row['사고자학년'] == '2학년'):
        return 18
    elif (row['학교급'] == '고등학교') and (row['사고자학년'] == '3학년'):
        return 19
    
    
    
df_new['나이'] = df_new.apply(make_age, axis=1)
df_new['나이'].astype(int)


28         7
33         7
38         7
39         7
41         7
          ..
193157    18
193160    17
193161    19
193166    19
193169    17
Name: 나이, Length: 609988, dtype: int32

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 614601 entries, 0 to 193176
Data columns (total 23 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   구분        614601 non-null  object
 1   학교급       614601 non-null  object
 2   지역        614601 non-null  object
 3   교육청       614601 non-null  object
 4   설립유형      614601 non-null  object
 5   사고자구분     614601 non-null  object
 6   사고자성별     614601 non-null  object
 7   사고자학년     614401 non-null  object
 8   사고발생일     614601 non-null  object
 9   사고발생요일    614601 non-null  object
 10  사고발생시각    614601 non-null  object
 11  사고시간      614601 non-null  object
 12  사고장소      614601 non-null  object
 13  사고부위      614601 non-null  object
 14  사고형태      614601 non-null  object
 15  사고당시활동    614601 non-null  object
 16  사고매개물     614601 non-null  object
 17  year      614601 non-null  int32 
 18  month     614601 non-null  int32 
 19  day       614601 non-null  int32 
 20  hour      614601 non-null  int3